In [4]:
import tensorflow as tf
from tensorflow.keras import mixed_precision

mixed_precision.set_global_policy('mixed_float16')

USING_LARGE = True

TRAIN_IMGS_DIRECTORY_SMALL = "data-small/train/"
VALIDATION_IMGS_DIRECTORY_SMALL = "data-small/valid/"

TRAIN_IMGS_DIRECTORY_LARGE = "data-large/train/"
VALIDATION_IMGS_DIRECTORY_LARGE = "data-large/valid/"
TEST_IMGS_DIRECTORY_LARGE = "data-large/valid/"

RESCALING_FACTOR = 1./0xFF
IMAGE_SIZE = (254, 254)
BATCH_SIZE = 8

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4050 Laptop GPU, compute capability 8.9


2024-05-01 10:27:10.591717: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


In [9]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img# data augmentation
import os

data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

def save_augs(src_dir, dest_dir, count_aug, max_to_aug):

        done = 1
        directory = os.fsencode(src_dir)
        
        for file in os.listdir(directory):
                if ~os.fsdecode(file).startswith("aug"):
                        img = load_img(f"{src_dir}/{os.fsdecode(file)}")  # this is a PIL image
                        x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
                        x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

                        # the .flow() command below generates batches of randomly transformed images
                        # and saves the results to the `preview/` directory
                        if done > max_to_aug:
                                break
                        i = 1
                        for batch in data_gen.flow(x, batch_size=1,
                                                save_to_dir=dest_dir, save_prefix='aug', save_format='jpg'):
                                i += 1
                                if i > count_aug:
                                        break  # otherwise the generator would loop indefinitely
                                
                        done += 1
                
#save_augs(f"{TRAIN_IMGS_DIRECTORY_LARGE}fake", f"{TRAIN_IMGS_DIRECTORY_LARGE}fake", 4, 2500)

In [10]:

# seems like shuffle has a negative effect on RAM usage and may cause OOM
if USING_LARGE:
    
    image_gen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=RESCALING_FACTOR,
    )
    
    train_dataset = image_gen.flow_from_directory(
        directory = TRAIN_IMGS_DIRECTORY_LARGE,
        target_size = IMAGE_SIZE,
        class_mode = "binary",
        seed = 0,
        shuffle = True,
        batch_size = BATCH_SIZE
    )

    validation_dataset = image_gen.flow_from_directory(
        directory = VALIDATION_IMGS_DIRECTORY_LARGE,
        target_size = IMAGE_SIZE,
        class_mode = "binary",
        shuffle = False,
        seed = 0,
        batch_size = BATCH_SIZE
    ) 


    test_dataset = image_gen.flow_from_directory(
        directory = TEST_IMGS_DIRECTORY_LARGE,
        target_size = IMAGE_SIZE,
        class_mode = "binary",
        shuffle = True,
        seed = 0,
        batch_size = BATCH_SIZE
    ) 
else:
    image_gen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=RESCALING_FACTOR,
        validation_split=0.15
    )
    train_dataset =  image_gen.flow_from_directory(
        directory = TRAIN_IMGS_DIRECTORY_SMALL,
        target_size = IMAGE_SIZE,
        class_mode = "binary",
        subset="training",
        shuffle = True,
        seed = 0,
        batch_size = BATCH_SIZE
    )

    validation_dataset =  image_gen.flow_from_directory(
        directory = TRAIN_IMGS_DIRECTORY_SMALL,
        target_size = IMAGE_SIZE,
        class_mode = "binary",
        subset="validation",
        shuffle = True,
        seed = 0,
        batch_size = BATCH_SIZE
    )

    test_dataset = image_gen.flow_from_directory(
        directory = VALIDATION_IMGS_DIRECTORY_SMALL,
        target_size = IMAGE_SIZE,
        class_mode = "binary",
        seed = 0,
        batch_size = BATCH_SIZE
    )

Found 115084 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


In [11]:
import matplotlib.pyplot as plt 
import tensorflow as tf

def visualize_image_dataset(dataset: tf.data.Dataset):
    plt.figure()
    class_names = dataset.class_names
    
    for images, labels in dataset.take(1):
        for ind in range(min(BATCH_SIZE, 6)):
            ax = plt.subplot(3, 3, ind + 1)
            plt.imshow(images[ind].numpy().astype("uint8"))
            plt.title(class_names[int(labels[ind])])
            plt.axis("off")
            
# visualize_image_dataset(train_dataset)

In [12]:
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization

INPUT_SHAPE = (*IMAGE_SIZE, 3)

model = tf.keras.models.Sequential([    
    tf.keras.Input(shape=(*IMAGE_SIZE, 3)),
    tf.keras.applications.DenseNet121(weights="imagenet/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5", include_top=False),
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(64, activation = "relu"),
    Dense(1, activation = "sigmoid")
])


In [13]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name="acc"),
        tf.keras.metrics.FalseNegatives(),
        tf.keras.metrics.FalsePositives(),
    ],
)


model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, None, None, 1024   7037504   
                             )                                   
                                                                 
 global_average_pooling2d_1  (None, 1024)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_3 (Dense)             (None, 512)               524800    
                                                                 
 batch_normalization_1 (Bat  (None, 512)               2048      
 chNormalization)                                                
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                      

In [14]:
from tensorflow.keras.saving import save_model

early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                     patience=5,
                                                     restore_best_weights=True,
                                                    )




reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                 factor=0.2,
                                                 patience=3)




early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                     patience=5,
                                                     restore_best_weights=True,
                                                    )


checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath="checkpoints",
    save_weights_only=False,
    monitor='val_loss',
    mode='min', #minimize the loss value
    save_best_only=True)

history = model.fit(
    train_dataset,
    epochs = 16,
    validation_data=validation_dataset,
    callbacks=[early_stopping_cb,reduce_lr]
)



Epoch 1/16
14386/14386 [==============================] - 1275s 86ms/step - loss: 0.6469 - acc: 0.6193 - false_negatives_1: 20753.0000 - false_positives_1: 23060.0000 - val_loss: 0.5059 - val_acc: 0.7471 - val_false_negatives_1: 2302.0000 - val_false_positives_1: 2756.0000 - lr: 0.0010
Epoch 2/16
14386/14386 [==============================] - 1242s 86ms/step - loss: 0.4713 - acc: 0.7792 - false_negatives_1: 11738.0000 - false_positives_1: 13675.0000 - val_loss: 0.3866 - val_acc: 0.8388 - val_false_negatives_1: 2022.0000 - val_false_positives_1: 1203.0000 - lr: 0.0010
Epoch 3/16
14386/14386 [==============================] - 1235s 86ms/step - loss: 0.3454 - acc: 0.8492 - false_negatives_1: 8126.0000 - false_positives_1: 9229.0000 - val_loss: 0.3219 - val_acc: 0.8641 - val_false_negatives_1: 1905.0000 - val_false_positives_1: 812.0000 - lr: 0.0010
Epoch 4/16
14386/14386 [==============================] - 1239s 86ms/step - loss: 0.2533 - acc: 0.8939 - false_negatives_1: 5502.0000 - false_

In [ ]:
model.predict(validation_dataset)

2500/2500 [==============================] - 70s 27ms/step


array([[0.5171889 ],
       [0.01223535],
       [0.2983586 ],
       ...,
       [0.7477453 ],
       [0.9935661 ],
       [0.9685962 ]], dtype=float32)

In [21]:
model.evaluate(test_dataset)

2500/2500 [==============================] - 79s 31ms/step - loss: 0.3877 - acc: 0.9020 - f1_score: 0.9084


[0.38767772912979126, 0.9020000100135803, array([0.9083513], dtype=float32)]

In [1]:
from tensorflow.keras.saving import load_model

model = load_model(
    "current_best.keras",
)

2024-05-01 10:26:05.522059: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-01 10:26:05.547052: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-01 10:26:05.547080: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-01 10:26:05.547720: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-01 10:26:05.551776: I tensorflow/core/platform/cpu_feature_guar

In [2]:
import cv2
import numpy as np

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [9]:
cap = cv2.VideoCapture(0)

IMAGE_SIZE = (254, 254)
def isRealLabel(isReal):
    if isReal:
        return "Real"
    else:
        return "AI"
    
try:
    isReal = True
    framelist = []
    
    while True:
        ret, frame = cap.read()
        

        if not ret:
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(32, 32), maxSize=IMAGE_SIZE)

        for (x, y, w, h) in faces:
            
            y_padding, x_padding = IMAGE_SIZE[1] - h, IMAGE_SIZE[0] - w
            y_st, x_st = int(y - y_padding / 2), int(x - x_padding / 2)
            y_end, x_end = int(y + h + y_padding / 2), int(x + w + x_padding / 2)
            
            cv2.rectangle(frame, (x_st, y_st), (x_end, y_end), (255, 0, 0), 2)
            
            face_frame = np.asarray(frame[y_st: y_end, x_st: x_end] * RESCALING_FACTOR)
            
            framelist.append(face_frame)

        cv2.putText(frame, isRealLabel(isReal), (x_st, y_st), cv2.FONT_HERSHEY_DUPLEX, 1, [0, 0, 255], 2)
        cv2.imshow('Live Video Stream', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        if len(framelist) >= BATCH_SIZE:
            mean_score = np.mean(model.predict(np.asarray(framelist), verbose = 0))
            print(mean_score)
            isReal = mean_score > 0.5
            framelist = []
            

        
finally:
    cap.release()
    cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0xae1af40) is not the object's thread (0x99df9e0).
Cannot move to target thread (0xae1af40)

QObject::moveToThread: Current thread (0xae1af40) is not the object's thread (0x99df9e0).
Cannot move to target thread (0xae1af40)

QObject::moveToThread: Current thread (0xae1af40) is not the object's thread (0x99df9e0).
Cannot move to target thread (0xae1af40)

QObject::moveToThread: Current thread (0xae1af40) is not the object's thread (0x99df9e0).
Cannot move to target thread (0xae1af40)

QObject::moveToThread: Current thread (0xae1af40) is not the object's thread (0x99df9e0).
Cannot move to target thread (0xae1af40)

QObject::moveToThread: Current thread (0xae1af40) is not the object's thread (0x99df9e0).
Cannot move to target thread (0xae1af40)

QObject::moveToThread: Current thread (0xae1af40) is not the object's thread (0x99df9e0).
Cannot move to target thread (0xae1af40)

QObject::moveToThread: Current thread (0xae1af40) is not the object's thread

0.98364747
0.9806667
0.9483838
0.94586265
0.97681695
0.92794764
0.9620043
0.9670613
0.9785801
0.9899089
0.9935664
0.9730526
0.9761007
0.9718313
0.9598841
0.9769754
0.9926524
0.9917178
0.98672205
0.98854977
0.9888177
0.991532
0.9923935
0.9879007
0.9930081
0.99545264
0.9954022
0.9940804


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (8,) + inhomogeneous part.